In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/arcane/')

from src._utils import load_bertopic_model_from_hf

/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-pac

In [2]:
with initialize(config_path="../../conf"):
    # Compose the configuration
    cfg = compose(config_name="ClustersResetService.yaml")

In [3]:
from src.clustering.ClustersResetService import ClustersResetService

In [4]:
clusters = ClustersResetService(cfg=cfg, random_state=186)

fetched all embeddings
proceeding to fetch article details
article details fetched
bertopic model instantiated


2023-11-02 22:11:49,245 - BERTopic - Reduced dimensionality
2023-11-02 22:11:49,283 - BERTopic - Clustered reduced embeddings


model fit to documents
docs assigned to balanced clusters


In [5]:
from src.clustering.ClusterHierarchyService import ClusterHierarchyService

In [20]:
clusters.bertopic_model.save('./sample_bert_pickle', serialization='pickle', save_embedding_model=False, save_ctfidf=False)

In [23]:
from bertopic import BERTopic

In [26]:
test_bert = BERTopic.load('./sample_bert_pickle', embedding_model='BAAI/bge-large-en-v1.5')

In [28]:
test_bert.transform('the indian stock market is expected to raise to new heights on the back of some strong headwinds')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-11-02 22:27:54,932 - BERTopic - Reduced dimensionality
2023-11-02 22:27:54,944 - BERTopic - Calculated probabilities with HDBSCAN
2023-11-02 22:27:54,944 - BERTopic - Predicted clusters


([-1],
 array([[0.00318604, 0.00107615, 0.00945971, 0.00465737, 0.00577817,
         0.00176121, 0.05653168, 0.00324656, 0.07563457, 0.04193103,
         0.00586909, 0.00503704, 0.00568083, 0.09738056, 0.06033816,
         0.0020334 , 0.09998431, 0.05755583, 0.07887946, 0.06866121,
         0.00566401, 0.05913563, 0.00229956]]))

In [33]:
os.environ['AWS_SECRET_KEY']

'0xslC7vXhuBeXurqbjTK6SSh8Mg/G4Z2EhwGq89i'

In [38]:
import boto3
import pickle
import io

def write_pickle_to_s3(bucket, folder, filename, data):
    s3_resource = boto3.resource('s3', aws_access_key_id=os.environ['AWS_ACCESS_KEY'], aws_secret_access_key=os.environ['AWS_SECRET_KEY'])
    pickle_byte_obj = pickle.dumps(data)
    s3_resource.Object(bucket, f'{folder}/{filename}').put(Body=pickle_byte_obj)

def read_pickle_from_s3(bucket, folder, filename):
    s3_resource = boto3.resource('s3', aws_access_key_id=os.environ['AWS_ACCESS_KEY'], aws_secret_access_key=os.environ['AWS_SECRET_KEY'])
    response = s3_resource.Object(bucket, f'{folder}/{filename}').get()
    body_string = response['Body'].read()
    return pickle.loads(body_string)

In [39]:
write_pickle_to_s3(bucket='insight-ml-models', folder='bertopic', filename = 'test_bertopic.pickle', data = clusters.bertopic_model)

In [40]:
other_test_bertopic = read_pickle_from_s3(bucket='insight-ml-models', folder='bertopic', filename = 'test_bertopic.pickle')

In [42]:
other_test_bertopic.transform('something something and something')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-11-02 22:51:37,357 - BERTopic - Reduced dimensionality
2023-11-02 22:51:37,363 - BERTopic - Calculated probabilities with HDBSCAN
2023-11-02 22:51:37,363 - BERTopic - Predicted clusters


([-1],
 array([[2.50464734e-05, 7.18825912e-06, 1.45117298e-04, 4.47661039e-05,
         5.31919597e-05, 1.14621164e-05, 5.50390843e-03, 2.53876232e-05,
         6.88558988e-03, 2.69151909e-03, 5.64223710e-05, 4.88016298e-05,
         5.50196806e-05, 9.44719463e-03, 5.42663634e-03, 1.33223139e-05,
         5.39284692e-01, 5.63892282e-03, 5.28979706e-02, 7.37651361e-03,
         5.16288679e-05, 5.23988897e-03, 1.59312981e-05]]))

In [48]:
def save_bertopic_model_to_s3(model: BERTopic, run_id: str):
    filename = 'BERTopic_' + run_id
    model.embedding_model = None
    s3_resource = boto3.resource('s3', aws_access_key_id=os.environ['AWS_ACCESS_KEY'], aws_secret_access_key=os.environ['AWS_SECRET_KEY'])
    pickle_byte_obj = pickle.dumps(model)
    s3_resource.Object(s3_bucket, f'{s3_bertopic_folder}/{filename}').put(Body=pickle_byte_obj)

In [50]:
s3_bucket = 'insight-ml-models'
s3_bertopic_folder = 'bertopic'

In [51]:
save_bertopic_model_to_s3(model=clusters.bertopic_model, run_id = 'sample_test_run')

In [63]:
def load_bertopic_model_from_s3(run_id: str, hf_embedding_model: str) -> BERTopic:
    filename = 'BERTopic_' + run_id
    s3_resource = boto3.resource('s3', aws_access_key_id=os.environ['AWS_ACCESS_KEY'], aws_secret_access_key=os.environ['AWS_SECRET_KEY'])
    response = s3_resource.Object(s3_bucket, f'{s3_bertopic_folder}/{filename}').get()
    body_string = response['Body'].read()
    model = pickle.loads(body_string)
    assert isinstance(model, BERTopic), f"Filed to load model from S3. model {filename} is not of type BERTopic. "
    model.embedding_model = SentenceTransformerBackend(embedding_model=hf_embedding_model)
    return model

In [67]:
chs.hierarchy_df['sample'] = 1

In [70]:
chs.hierarchy_df['sample'].unique()[0]

1

In [64]:
sample_s3_bert = load_bertopic_model_from_s3('sample_test_run', hf_embedding_model='BAAI/bge-large-en-v1.5')

In [65]:
sample_s3_bert.transform('something something and something')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-11-02 23:08:32,204 - BERTopic - Reduced dimensionality
2023-11-02 23:08:32,210 - BERTopic - Calculated probabilities with HDBSCAN
2023-11-02 23:08:32,210 - BERTopic - Predicted clusters


([-1],
 array([[2.50464734e-05, 7.18825912e-06, 1.45117298e-04, 4.47661039e-05,
         5.31919597e-05, 1.14621164e-05, 5.50390843e-03, 2.53876232e-05,
         6.88558988e-03, 2.69151909e-03, 5.64223710e-05, 4.88016298e-05,
         5.50196806e-05, 9.44719463e-03, 5.42663634e-03, 1.33223139e-05,
         5.39284692e-01, 5.63892282e-03, 5.28979706e-02, 7.37651361e-03,
         5.16288679e-05, 5.23988897e-03, 1.59312981e-05]]))

In [44]:
from bertopic import BERTopic

def convert_to_bertopic(obj):
   # Create a new BERTopic object
    bertopic_obj = BERTopic()

   # Copy the attributes from the original object to the new object
    for attr, value in obj.__dict__.items():
        setattr(bertopic_obj, attr, value)

    return bertopic_obj


In [45]:
some_other_bertopic = convert_to_bertopic(other_test_bertopic)

In [47]:
some_other_bertopic.transform('something something and something')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-11-02 23:01:54,967 - BERTopic - Reduced dimensionality
2023-11-02 23:01:54,970 - BERTopic - Calculated probabilities with HDBSCAN
2023-11-02 23:01:54,971 - BERTopic - Predicted clusters


([-1],
 array([[2.50464734e-05, 7.18825912e-06, 1.45117298e-04, 4.47661039e-05,
         5.31919597e-05, 1.14621164e-05, 5.50390843e-03, 2.53876232e-05,
         6.88558988e-03, 2.69151909e-03, 5.64223710e-05, 4.88016298e-05,
         5.50196806e-05, 9.44719463e-03, 5.42663634e-03, 1.33223139e-05,
         5.39284692e-01, 5.63892282e-03, 5.28979706e-02, 7.37651361e-03,
         5.16288679e-05, 5.23988897e-03, 1.59312981e-05]]))

In [ ]:
import boto3
import pickle
import io

def write_pickle_to_s3(bucket, key, data):
    s3_resource = boto3.resource('s3')
    pickle_byte_obj = pickle.dumps(data)
    s3_resource.Object(bucket, key).put(Body=pickle_byte_obj)

def read_pickle_from_s3(bucket, key):
    s3_resource = boto3.resource('s3')
    response = s3_resource.Object(bucket, key).get()
    body_string = response['Body'].read()
    return pickle.loads(body_string)


In [6]:
chs = ClusterHierarchyService(documents=clusters.documents, bertopic_model=clusters.bertopic_model, cfg = clusters.cfg)

100%|██████████| 22/22 [00:00<00:00, 106.90it/s]


In [7]:
chs._format_hierarchy_df()

In [8]:
chs._get_all_child_ids()

In [9]:
chs._perform_validations()

In [10]:
chs._find_root_cluster_id()

In [11]:
chs._add_leaf_topics_to_hierarchy_df()

In [12]:
chs.hierarchy_df

,cluster_id,cluster_name,storylines,left_child_id,left_child_name,right_child_id,right_child_name,distance
0,44,estate_real estate_card_tax_credit card,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",42,estate_real estate_property_residential_commer...,43,card_tax_credit card_cards_section,1.321550
1,43,card_tax_credit card_cards_section,"[0, 3, 4, 7, 10, 11, 12, 20, 22]",34,card_credit card_cards_credit cards_score,36,tax_section_bonds_income tax_risk,1.107288
2,42,estate_real estate_property_residential_commer...,"[1, 2, 5, 6, 8, 9, 13, 14, 15, 16, 17, 18, 19,...",30,series_season_episode_october_episodes,41,estate_real estate_property_residential_commer...,1.092794
3,41,estate_real estate_property_residential_commer...,"[1, 2, 6, 8, 9, 13, 14, 16, 17, 18, 19, 21]",32,cent_quarter_coal_steel_inflation,40,estate_real estate_property_residential_commer...,1.050642
4,40,estate_real estate_property_residential_commer...,"[1, 2, 6, 14, 17, 18, 19, 21]",39,estate_real estate_property_residential_commer...,2,live blog_live_blog_mint_52 week,1.010815
5,39,estate_real estate_property_residential_commer...,"[1, 6, 14, 17, 18, 19, 21]",31,estate_real estate_property_residential_commer...,38,adani_dicgc_cr_crs_israel,0.991466
6,38,adani_dicgc_cr_crs_israel,"[6, 14, 17, 19, 21]",37,adani_cr_crs_israel_sebi,21,dicgc_depositors_deposits_deposit insurance_in...,0.987865
7,37,adani_cr_crs_israel_sebi,"[6, 14, 17, 19]",29,adani_energy_renewable_billion_usd,35,crs_israel_cr_sebi_edelweiss,0.946517
8,36,tax_section_bonds_income tax_risk,"[3, 4, 10, 11, 12]",28,tax_section_bonds_income tax_deduction,26,strategy_ipo_traders_strike_underlying,0.943983
9,35,crs_israel_cr_sebi_edelweiss,"[6, 14]",6,israel_sebi_court_gaza_spicejet,14,crs_cr_edelweiss_march 31_ncd,0.939743


In [13]:
chs._populate_levels(parent_cluster_id=chs.root_cluster_id, parent_level=0)

levels are {23: 4} for parent cluster id 23. the left id is 21       4
45    None
Name: left_child_id, dtype: object
levels are {28: 3} for parent cluster id 28. the left id is 23


{44: 0,
 42: 1,
 30: 2,
 5: 3,
 15: 3,
 41: 2,
 32: 3,
 27: 4,
 13: 5,
 16: 5,
 25: 4,
 8: 5,
 9: 5,
 40: 3,
 39: 4,
 31: 5,
 1: 6,
 18: 6,
 38: 5,
 37: 6,
 29: 7,
 19: 8,
 17: 8,
 35: 7,
 6: 8,
 14: 8,
 21: 6,
 2: 4,
 43: 1,
 34: 2,
 24: 3,
 7: 4,
 0: 4,
 33: 3,
 22: 4,
 20: 4,
 36: 2,
 28: 3,
 10: 4,
 26: 3,
 3: 4,
 11: 4}

In [15]:
dct = {}
for i in range(-1, len(clusters.bertopic_model.topic_embeddings_)-1):
    dct[i] = clusters.bertopic_model.topic_embeddings_[i+1]

In [26]:
from src.clustering.ClusterHierarchyService import ClusterHierarchyColumns

In [19]:
leaf_topic_rows = [{ClusterHierarchyColumns.cluster_id: topic_id,
                            ClusterHierarchyColumns.cluster_name: topic_name,
                            ClusterHierarchyColumns.left_child_id: None,
                            ClusterHierarchyColumns.left_child_name: '',
                            ClusterHierarchyColumns.right_child_id: None,
                            ClusterHierarchyColumns.right_child_name: '',
                            ClusterHierarchyColumns.storylines: []} for topic_id, topic_name in enumerate(clusters.bertopic_model.topic_labels_)]

In [16]:
chs.hierarchy_df

,cluster_id,cluster_name,storylines,left_child_id,left_child_name,right_child_id,right_child_name,distance
0,42,estate_card_real estate_price_commercial,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",40,estate_real estate_commercial_property_residen...,41,card_credit card_price_cards_options,1.305643
1,41,card_credit card_price_cards_options,"[0, 3, 4, 8, 11, 13, 17, 18, 21]",31,price_options_bonds_risk_strategy,34,card_credit card_cards_section_credit cards,1.126501
2,40,estate_real estate_commercial_property_residen...,"[1, 2, 5, 6, 7, 9, 10, 12, 14, 15, 16, 19, 20]",39,live_live blog_blog_series_season,38,estate_real estate_commercial_property_residen...,1.106256
3,39,live_live blog_blog_series_season,"[1, 6, 14, 20]",29,series_season_release_episode_episodes,37,live blog_live_blog_mint_52 week,1.018431
4,38,estate_real estate_commercial_property_residen...,"[2, 5, 7, 9, 10, 12, 15, 16, 19]",27,cent_quarter_coal_steel_sales,36,estate_real estate_commercial_property_residen...,1.004355
5,37,live blog_live_blog_mint_52 week,"[1, 20]",20,israel_gaza_hamas_blinken_civilians,1,live blog_live_blog_mint_52 week,0.993697
6,36,estate_real estate_commercial_property_residen...,"[2, 9, 10, 15, 16, 19]",30,estate_real estate_commercial_property_residen...,35,adani_cr_crs_energy_edelweiss,0.983176
7,35,adani_cr_crs_energy_edelweiss,"[9, 10, 16, 19]",28,adani_energy_power_renewable_billion,33,cr_crs_edelweiss_sebi_march 31,0.958677
8,34,card_credit card_cards_section_credit cards,"[0, 4, 17, 18, 21]",24,section_income tax_pan_deduction_act,32,card_credit card_cards_credit cards_score,0.938703
9,33,cr_crs_edelweiss_sebi_march 31,"[9, 10]",10,cr_crs_edelweiss_march 31_ncd,9,spicejet_court_resolution_sebi_airline,0.923644


In [24]:
import pandas as pd

In [29]:
def _populate_levels(parent_cluster_id, parent_level):
    # notice that not all leaf clusters: stories are at the same level.
    if pd.isna(parent_cluster_id) or parent_cluster_id is None:  # Using pd.na because Nones are converted to pd.na by pandas dataframe
        return {}

    levels = {parent_cluster_id: parent_level}
    cluster_row = chs.hierarchy_df[chs.hierarchy_df[ClusterHierarchyColumns.cluster_id] == parent_cluster_id].squeeze()
    if not cluster_row.empty:  # We already asserted above that a cluster has a max of 1 row
        left_child_id = cluster_row[ClusterHierarchyColumns.left_child_id]
        right_child_id = cluster_row[ClusterHierarchyColumns.right_child_id]
        try:
            left_level = _populate_levels(left_child_id, parent_level + 1)
            levels.update(left_level)
        except:
            print(f'levels are {levels} for parent cluster id {parent_cluster_id}. the left id is {left_child_id}')
        levels.update(_populate_levels(right_child_id, parent_level + 1))
    return levels

In [30]:
_populate_levels(chs.root_cluster_id,0)

{34: 0,
 22: 1,
 18: 2,
 10: 3,
 11: 3,
 9: 2,
 33: 1,
 30: 2,
 3: 3,
 25: 3,
 12: 4,
 4: 4,
 32: 2,
 29: 3,
 27: 4,
 2: 5,
 23: 5,
 0: 6,
 7: 6,
 24: 4,
 13: 5,
 21: 5,
 15: 6,
 8: 6,
 31: 3,
 26: 4,
 20: 5,
 16: 6,
 5: 6,
 17: 5,
 28: 4,
 19: 5,
 6: 6,
 1: 6,
 14: 5}

In [10]:
chs.hierarchy_df

,cluster_id,cluster_name,storylines,left_child_id,left_child_name,right_child_id,right_child_name,distance
20,42,estate_card_real estate_price_commercial,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",40,estate_real estate_commercial_property_residen...,41,card_credit card_price_cards_options,1.305643
19,41,card_credit card_price_cards_options,"[0, 3, 4, 8, 11, 13, 17, 18, 21]",31,price_options_bonds_risk_strategy,34,card_credit card_cards_section_credit cards,1.126501
18,40,estate_real estate_commercial_property_residen...,"[1, 2, 5, 6, 7, 9, 10, 12, 14, 15, 16, 19, 20]",39,live_live blog_blog_series_season,38,estate_real estate_commercial_property_residen...,1.106256
17,39,live_live blog_blog_series_season,"[1, 6, 14, 20]",29,series_season_release_episode_episodes,37,live blog_live_blog_mint_52 week,1.018431
16,38,estate_real estate_commercial_property_residen...,"[2, 5, 7, 9, 10, 12, 15, 16, 19]",27,cent_quarter_coal_steel_sales,36,estate_real estate_commercial_property_residen...,1.004355
15,37,live blog_live_blog_mint_52 week,"[1, 20]",20,israel_gaza_hamas_blinken_civilians,1,live blog_live_blog_mint_52 week,0.993697
14,36,estate_real estate_commercial_property_residen...,"[2, 9, 10, 15, 16, 19]",30,estate_real estate_commercial_property_residen...,35,adani_cr_crs_energy_edelweiss,0.983176
13,35,adani_cr_crs_energy_edelweiss,"[9, 10, 16, 19]",28,adani_energy_power_renewable_billion,33,cr_crs_edelweiss_sebi_march 31,0.958677
12,34,card_credit card_cards_section_credit cards,"[0, 4, 17, 18, 21]",24,section_income tax_pan_deduction_act,32,card_credit card_cards_credit cards_score,0.938703
11,33,cr_crs_edelweiss_sebi_march 31,"[9, 10]",10,cr_crs_edelweiss_march 31_ncd,9,spicejet_court_resolution_sebi_airline,0.923644
